<!-- ![](./imgs/局部坐标系-全局坐标系.png) -->

<img src="./imgs/局部坐标系-全局坐标系.png" width="500" height="auto">

当局部坐标系的z轴确定时，无论x轴和y轴如何，局部坐标系内dxz+dyz轨道保持不变

全局坐标系用下标b，局部坐标系用下标e，

坐标用 $r$ 表示，系数用 $c$ 表示

$r_b$是全局坐标系中的点在局部坐标系中的描述（相对位置）

局部坐标用矩阵 $T$ 表示，每一列是局部坐标系的一个轴，形状为`[3,3]`

系数的变换矩阵用矩阵 $M$ 表示，形状为`[6,6]`



$r_b@T=r_e$

$r_e@T^{-1}=r_b$

$M_{be}@c_b=c_e$

$M_{eb}@c_e=c_b$

In [9]:
from sympy import symbols, expand, Matrix, latex
from sympy.core.add import Add
import sympy as sp

# 定义符号变量
xb, yb, zb = symbols('x_b y_b z_b')
xe, ye, ze = symbols('x_e y_e z_e')
T = sp.Matrix(symbols('T11 T12 T13 T21 T22 T23 T31 T32 T33')).reshape(3,3) # 局部坐标系
# sin,cos= symbols('sin cos')
# T=sp.Matrix([
#     [cos,-sin,0],
#     [sin,cos, 0],
#     [0,0,     1]
# ])

def get_Mbe(): # 全局坐标系到局部坐标系的变换矩阵
    # 用xb表示xe
    xe = T[0,0]*xb + T[1,0]*yb + T[2,0]*zb  # x' = T11*x + T21*y + T31*z
    ye = T[0,1]*xb + T[1,1]*yb + T[2,1]*zb  # y' = T12*x + T22*y + T32*z
    ze = T[0,2]*xb + T[1,2]*yb + T[2,2]*zb  # z' = T13*x + T23*y + T33*z
    basis_B = [xb*xb, yb*yb, zb*zb,xb*yb, xb*zb, yb*zb]
    basis_E = [xe*xe, ye*ye, ze*ze,xe*ye, xe*ze, ye*ze]
    MD = []
    for term_E in basis_E:
        expanded_term:Add = expand(term_E)
        row = [expanded_term.coeff(term_B) for term_B in basis_B]
        MD.append(row)

    MD = Matrix(MD).T
    return MD

def get_Meb(): # 局部坐标系到全局坐标系的变换矩阵
    # 用xe表示xb
    xb = T[0,0]*xe + T[0,1]*ye + T[0,2]*ze
    yb = T[1,0]*xe + T[1,1]*ye + T[1,2]*ze
    zb = T[2,0]*xe + T[2,1]*ye + T[2,2]*ze
    basis_B = [xb*xb, yb*yb, zb*zb,xb*yb, xb*zb, yb*zb] # 每一项都是re的线性组合，找到组合系数
    basis_E = [xe*xe, ye*ye, ze*ze,xe*ye, xe*ze, ye*ze]
    MD = []
    for term_B in basis_B:
        expanded_term:Add = expand(term_B) # 展开成加法项
        row = [expanded_term.coeff(term_E) for term_E in basis_E]
        MD.append(row)
    MD = Matrix(MD).T
    return MD

get_Meb()

Matrix([
[   T11**2,    T21**2,    T31**2,           T11*T21,           T11*T31,           T21*T31],
[   T12**2,    T22**2,    T32**2,           T12*T22,           T12*T32,           T22*T32],
[   T13**2,    T23**2,    T33**2,           T13*T23,           T13*T33,           T23*T33],
[2*T11*T12, 2*T21*T22, 2*T31*T32, T11*T22 + T12*T21, T11*T32 + T12*T31, T21*T32 + T22*T31],
[2*T11*T13, 2*T21*T23, 2*T31*T33, T11*T23 + T13*T21, T11*T33 + T13*T31, T21*T33 + T23*T31],
[2*T12*T13, 2*T22*T23, 2*T32*T33, T12*T23 + T13*T22, T12*T33 + T13*T32, T22*T33 + T23*T32]])

In [10]:
get_Mbe()

Matrix([
[   T11**2,    T12**2,    T13**2,           T11*T12,           T11*T13,           T12*T13],
[   T21**2,    T22**2,    T23**2,           T21*T22,           T21*T23,           T22*T23],
[   T31**2,    T32**2,    T33**2,           T31*T32,           T31*T33,           T32*T33],
[2*T11*T21, 2*T12*T22, 2*T13*T23, T11*T22 + T12*T21, T11*T23 + T13*T21, T12*T23 + T13*T22],
[2*T11*T31, 2*T12*T32, 2*T13*T33, T11*T32 + T12*T31, T11*T33 + T13*T31, T12*T33 + T13*T32],
[2*T21*T31, 2*T22*T32, 2*T23*T33, T21*T32 + T22*T31, T21*T33 + T23*T31, T22*T33 + T23*T32]])

In [11]:
cxz,cyz=symbols(fr'c_{{xz}} c_{{yz}}')
cbs=Matrix([0,0,0,0,cxz,cyz])
cbs

Matrix([
[     0],
[     0],
[     0],
[     0],
[c_{xz}],
[c_{yz}]])

In [12]:
Mbe=get_Mbe()
Mbe@cbs

Matrix([
[                        T11*T13*c_{xz} + T12*T13*c_{yz}],
[                        T21*T23*c_{xz} + T22*T23*c_{yz}],
[                        T31*T33*c_{xz} + T32*T33*c_{yz}],
[c_{xz}*(T11*T23 + T13*T21) + c_{yz}*(T12*T23 + T13*T22)],
[c_{xz}*(T11*T33 + T13*T31) + c_{yz}*(T12*T33 + T13*T32)],
[c_{xz}*(T21*T33 + T23*T31) + c_{yz}*(T22*T33 + T23*T32)]])